# Exercise 2: Evolution Strategy

## Import libraries

In [1]:
import numpy as np

np.set_printoptions(precision=4, suppress=True) # for better printing

# ES ($\mu$,$\lambda$) algorithm
In this exercise, we will use Evolution Strategies ($\mu$,$\lambda$) as explained in the lecture. The algorithm was introduced with the following steps:

- **Step 1:** Set up population size $\lambda$, number of parents $\mu$, mutation step size $\sigma$
- **Step 2:** Set up solution guess, e.g, $n\times 1$ vector $m$
- **Step 3:** Generate $\lambda$ offspring from $m$
- **Step 4:** Evaluate $\lambda$ offspring
- **Step 5:** Select $\mu$ parents with Truncated Rank Selection
- **Step 6:** Update population $m$ using fitness-weighted values of $\mu$ parents

- **Repeat from Step 3** until convergence or maximum number of generations

In the following, we will implement the algorithm. Step 4. is problem specific and already prepared. We will select the parameters of Step 1. at the end of the exercise for each problem.

## Step 2: Initialise $m$

-------
### **Assignment**
Implement the `initialise_mean`-function. Initialise the mean by using a random uniform distribution. The mean vector should have the same size as the number of parameters specified. Sample from a uniform distribution in range between -4.0 and 4.0 for the initial mean.

### **Hint**
To sample from a uniform distribution with NumPy, you can use `np.random.uniform`. Check the NumPy [documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html) for further details. 

-------

In [2]:
def initialise_mean(num_parameters):
    mean_vector = ...
    return mean_vector

In [3]:
# Solution
def initialise_mean(num_parameters):
    mean_vector = np.random.uniform(low=-4.0, high=4.0, size=(num_parameters,))
    return mean_vector

In [4]:
# Test your function:
num_parameters = 5
mean_vector = initialise_mean(num_parameters=num_parameters)

print(f"Mean shape: {mean_vector.shape}\n\nMean parameters:\n{mean_vector}")
assert len(mean_vector) == num_parameters

Mean shape: (5,)

Mean parameters:
[-0.2964 -0.3113 -3.1512 -3.3512  1.4983]


## Step 3: Generate $\lambda$ offspring from $m$

We introduce the following notation for storing the parameters of our population. The population parameters are stored in a 2D array with dimension (population_size, parameter_size). That means that each row corresponds to the parameters of one population member.

|   | Parameter 1   | Parameter 2   | $\cdots$   | Parameter n   |
|:---:|:---:|:---:|:---:|:---:|
| Offspring 1   | $\dots$  | $\dots$  |  $\dots$ |  $\dots$ |
| Offspring 2  | $\dots$  |  $\dots$ | $\dots$  |  $\dots$ |
| $\vdots$  | $\dots$ | $\dots$  | $\dots$  | $\dots$  |
| Offspring m  |  $\dots$ | $\dots$  |  $\dots$ | $\dots$  |

For generating a new offspring the lecture introduced the following formula:

\begin{align}
    x_i = m + \sigma \mathcal{N}_i(0, \mathcal{I}) \qquad \text{for} \quad 0 < i \le \lambda
\end{align}

---
### **Assignment**
Implement the `generate_offspring`-function with the formula given from the lecture to generate the population as specified above.

### **Hint**
You can generate multivariate (normal) Gaussian distributions with `np.random.normal`. Check the NumPy [documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html) for further details.

---

In [5]:
def generate_mutated_offspring(population_size, mean_vector, mutation_sigma):
    # Duplicate mean candidate along y-dim
    population = ...
    
    # Compute multivariate Gaussian noise
    mutation = ...
    
    # Compute offspring
    mutated_population = ...
    
    return mutated_population

In [6]:
# Solution
def generate_mutated_offspring(population_size, mean_vector, mutation_sigma):   
    # Duplicate mean candidate along y-dim
    population = np.tile(mean_vector,(population_size, 1))
    
    # Compute multivariate Gaussian noise
    num_parameters = len(mean_vector)
    perturbation = np.random.normal(
        loc=0.0, 
        scale=1.0, 
        size=(population_size, num_parameters)
    )
    
    # Compute offspring
    mutated_population = population + mutation_sigma * perturbation
    
    return mutated_population

In [7]:
# Test your function:
population_size = 8
mean_vector = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
mutation_sigma = 0.3

population = generate_mutated_offspring(population_size, mean_vector, mutation_sigma)
print(f"Population shape: {population.shape}")
print(f"Population params:\n{population}")

Population shape: (8, 5)
Population params:
[[1.1091 2.0506 2.8068 4.3635 5.3636]
 [0.8561 2.0598 2.6816 4.081  5.3522]
 [1.1508 1.9964 2.9869 4.1841 4.7886]
 [1.0584 1.9338 2.7807 4.8245 4.6777]
 [0.496  2.2752 2.6352 3.6575 5.2004]
 [1.2145 1.6065 2.793  4.4394 4.9926]
 [1.1369 1.9331 2.8824 4.3476 5.5869]
 [0.9652 2.1701 3.197  3.9579 4.6231]]


## Step 5: Select $\mu $ parents with Truncated Rank Selection

Following the evaluation of the population, a fitness vector is obtained. This vector is used to organize the population and select the optimum solutions as parents for the upcoming generation. The parents' population is represented as a 2D array with the shape (num_parents $\lambda$, num_parameters $n$). This process follows the ES ($\mu$,$\lambda$) Strategy, implying that the offspring completely replaces the previous generation after each cycle.

|   | Parameter 1   | Parameter 2   | $\cdots$   | Parameter n   |
|:---:|:---:|:---:|:---:|:---:|
| Parent 1   | $\dots$  | $\dots$  |  $\dots$ |  $\dots$ |
| Parent 2  | $\dots$  |  $\dots$ | $\dots$  |  $\dots$ |
| $\vdots$  | $\dots$ | $\dots$  | $\dots$  | $\dots$  |
| Parent $\mu$  |  $\dots$ | $\dots$  |  $\dots$ | $\dots$  |

---
### **Assignment**
Implement the function `sort_and_select_parents` to select the parents for the next generation based on the current population's fitness values.
1. **Sort the Population:** Arrange the population based on the provided fitness values in **descending** order.
2. **Truncate the Lower Performers:** Exclude population members that did not perform adequately.

Return the selected parents' population along with their corresponding fitness scores. Note that in the problems we aim to solve, a higher fitness score indicates better performance.

### **Hint**
NumPy provides a handy function for sorting an array callend `np.argsort`, for sorting an array, which returns the indices of the array in **ascending** order. Utilize these indices for **indexing** tto arrange the array. For more information, refer to the documentation on [argsort](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) and [indexing](https://numpy.org/doc/stable/user/basics.indexing.html), check the documentation. 

---

In [8]:
def sort_and_select_parents(population, fitness, num_parents):
    parent_population = ...
    parent_fitness = ...
    return parent_population, parent_fitness

In [9]:
# Solution
def sort_and_select_parents(population, fitness, num_parents):
    sorted_indices = np.argsort(fitness)[::-1]
    sorted_indices = sorted_indices[0:num_parents]
    
    parent_population = population[sorted_indices]
    parent_fitness = fitness[sorted_indices]
    
    return parent_population, parent_fitness

In [10]:
# Test your function:
population_size = 8
num_parameters = 5
num_parents = 4

population = np.random.normal(loc=1.0, size=(population_size, num_parameters))
fitness = np.random.uniform(size=(population_size,))

parent_population, parent_fitness = sort_and_select_parents(
    population=population, 
    fitness=fitness, 
    num_parents=num_parents
)

print(f"Population:\n{population}")
print(f"Population fitness:\n{fitness}")
print()
print(f"Parent population shape: {parent_population.shape}")
print(f"Parent population:\n{parent_population}")
print(f"Parent fitness:\n{parent_fitness}")

Population:
[[-0.9465  0.7613  1.3554  1.0655  0.0166]
 [ 2.7963  0.7238  2.5472  1.0708  2.5522]
 [ 1.9508  1.8885 -0.0832  2.7253  3.708 ]
 [ 0.1879 -0.0691  0.7854  2.7975  0.8841]
 [ 1.8424  0.4237  1.3386  0.0335  3.4204]
 [ 1.2768  1.5551  1.1647 -1.1492  2.2284]
 [ 0.5721  2.6282  1.6594  2.8103  0.4368]
 [ 1.0941  1.2336  1.2113  1.6071 -0.4625]]
Population fitness:
[0.29   0.6118 0.3828 0.3175 0.5708 0.3922 0.7617 0.048 ]

Parent population shape: (4, 5)
Parent population:
[[ 0.5721  2.6282  1.6594  2.8103  0.4368]
 [ 2.7963  0.7238  2.5472  1.0708  2.5522]
 [ 1.8424  0.4237  1.3386  0.0335  3.4204]
 [ 1.2768  1.5551  1.1647 -1.1492  2.2284]]
Parent fitness:
[0.7617 0.6118 0.5708 0.3922]


## Step 6: Update population $m$ using fitness-weighted values of $\mu$ parents

Once the parent population has been selected, the next step is to generate a new population mean $m$ by considering the fitness of each parent.

---
### **Assignment**
Implement the function `update_population_mean` to calculate the new population mean based on the fitness-weighted values of the parents.
1. **Fitness-Weight the Parent Fitness:** Adjust the fitness values of the parents to act as weights in the calculation of the new mean.
2. **Calculate Weighted Mean:** Determine the mean of all population members by considering these fitness-weighted values.


### **Hint**
The weighted mean can generally be computed using the formula:
\begin{align}
\mathbf{\mu} = \frac{\sum_{i=1}^{N} w_i \cdot \mathbf{x}_i}{\sum_{i=1}^{N} w_i}
\end{align}

with 
- $w_i$ represents the weights corresponding to the fitness values of each population member,
- $\mathbf{\mu}$ is the mean parameters of the current population,
- $\mathbf{x}$ denotes the parameters of each population member.

There are multiple ways of weighting the population parameters to the fitness scores. One way is using nested `for`-loops. Computationally more efficient is to use NumPy matrix multiplications like `np.outer`, `np.vstack` and `np.multiply`.

---

In [11]:
def update_population_mean(parent_population, parent_fitness):
    # Normalise parent fitness scores
    normed_parents_fitness = ...
    
    # Compute population weighted to the normed fitness scores  
    weighted_parents_population = ...
    
    # Calculate the sum of weighted parents population
    updated_mean_vector = ...
    
    return updated_mean_vector

In [12]:
# Solution: Without fitness-weighting
def update_population_mean(parent_population):    
    # Calculate mean of population along y-axis
    updated_mean_vector = np.mean(parent_population, axis=0)
    
    return updated_mean_vector

In [13]:
# Solution: Fitness-weighted
def update_population_mean(parent_population, parent_fitness):
    # Normalise parent fitness scores
    normed_parents_fitness = parent_fitness / np.sum(parent_fitness)
    
    # Compute population weighted to the normed fitness scores    
    weight = np.outer(normed_parents_fitness, np.ones((1, parent_population.shape[1])))
    weighted_parents_population = np.multiply(parent_population, weight) #hadamard product
    
    # Calculate mean of weighted parents population along y-axis
    updated_mean_vector = np.sum(weighted_parents_population, axis=0)
    
    return updated_mean_vector

In [14]:
# Test your function:
np.set_printoptions(precision=4, suppress=True)

num_parameters = 5
num_parents = 4

parent_population = np.random.normal(loc=1.0, scale=0.5, size=(num_parents, num_parameters))
parent_fitness = np.random.uniform(size=(num_parents,))

mean_vector = update_population_mean(parent_population, parent_fitness)

print("====== Small differences ======")
print(f"Best candidate:\n{parent_population[0,:]}")
print(f"Fitness-weighted mean:\n{mean_vector}")
print(f"Population mean unweighted:\n{np.mean(population, axis=0)}")
print()

# Assume one solution clearly outperforms the others.
parent_fitness[0] = 100.0
mean_vector = update_population_mean(parent_population, parent_fitness)

print("====== Large differences ======")
print(f"Best candidate:\n{parent_population[0,:]}")
print(f"Fitness-weighted mean:\n{mean_vector}")
print(f"Parents population mean unweighted:\n{np.mean(parent_population, axis=0)}")

====== Small differences ======
Best candidate:
[0.6261 0.6564 0.651  1.3742 1.4234]
Fitness-weighted mean:
[0.7998 0.7424 1.1604 1.0868 0.7028]
Population mean unweighted:
[1.0968 1.1431 1.2474 1.3701 1.598 ]

====== Large differences ======
Best candidate:
[0.6261 0.6564 0.651  1.3742 1.4234]
Fitness-weighted mean:
[0.6321 0.6594 0.6688 1.3641 1.3983]
Parents population mean unweighted:
[0.7908 0.7543 1.1395 1.1033 0.7151]
